<a href="https://colab.research.google.com/github/jessedeveloperinvestor/AI-ML-Cheatsheets/blob/main/Big_Query.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ☁️ Cloud Computing com Google Colab + GCP

### 🔍 Coleta de Citações via API + Armazenamento no Cloud Storage + Ingestão no BigQuery

**Autor:** Fernando Nakamuta  
**Projeto:** poli-usp-pro-cloud-computing  
**Dataset:** `poliusp_demo.quotes`  
**Serviços utilizados:** `requests`, `Cloud Storage`, `BigQuery`

# Utilizando BigQuery


In [ ]:
# Imports
from google.colab import auth
from google.cloud import bigquery
from google.cloud import storage

## Autenticação no Google Cloud

In [ ]:
auth.authenticate_user()

In [ ]:
project_id = "poli-usp-pro-cloud-computing"
bq_client = bigquery.Client(project=project_id)

## Carregando dados do Cloud Storage para o BigQuery

In [ ]:
# Tabela de destino
dataset_id = "poliusp_demo"           # <-- troque para seu dataset
table_id = "scrape_quotes"                  # nome da nova tabela
table_ref = f"{project_id}.{dataset_id}.{table_id}"

In [ ]:
dataset_id = f"{project_id}.poliusp_demo"

# Cria o dataset se não existir
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"  # ou "SOUTHAMERICA-EAST1" se preferir
dataset = bq_client.create_dataset(dataset, exists_ok=True)

print(f"Dataset {dataset.dataset_id} criado com sucesso!")

Dataset poliusp_demo criado com sucesso!


In [ ]:
from google.cloud import bigquery


gcs_uri = f"gs://poliusp-pro-exemplo-aula/quotes/*.json"


# Configuração do job de carga
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True,  # Detecta esquema automaticamente
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # sobrescreve se já existir
)


# Inicia o job de carga
load_job = bq_client.load_table_from_uri(
    gcs_uri,
    destination=table_ref,
    job_config=job_config
)


load_job.result()  # Espera o job finalizar

LoadJob<project=poli-usp-pro-cloud-computing, location=US, id=0e9c1329-3038-4fe1-9208-9854b8828bca>

In [ ]:
# Verifica a tabela criada
table = bq_client.get_table(table_ref)
print(f"Tabela {table.table_id} carregada com {table.num_rows} linhas.")

Tabela scrape_quotes carregada com 100 linhas.


## Lendo dados do BigQuery

In [ ]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id)

query = """
SELECT author, text
FROM `poli-usp-pro-cloud-computing.poliusp_demo.scrape_quotes`
WHERE author.name = 'Albert Einstein'
LIMIT 10
"""

df = bq_client.query(query).to_dataframe()

,author,text
0,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...","“If you can't explain it to a six year old, yo..."
1,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...","“If you want your children to be intelligent, ..."
2,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“Logic will get you from A to Z; imagination w...
3,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“Any fool can know. The point is to understand.”
4,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“Life is like riding a bicycle. To keep your b...
5,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...","“If I were not a physicist, I would probably b..."
6,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“Anyone who has never made a mistake has never...
7,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“The world as we have created it is a process ...
8,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“There are only two ways to live your life. On...
9,"{'name': 'Albert Einstein', 'slug': 'Albert-Ei...",“Try not to become a man of success. Rather be...


In [ ]:
for text in df["text"]:
    print(text)

“If you can't explain it to a six year old, you don't understand it yourself.”
“If you want your children to be intelligent, read them fairy tales. If you want them to be more intelligent, read them more fairy tales.”
“Logic will get you from A to Z; imagination will get you everywhere.”
“Any fool can know. The point is to understand.”
“Life is like riding a bicycle. To keep your balance, you must keep moving.”
“If I were not a physicist, I would probably be a musician. I often think in music. I live my daydreams in music. I see my life in terms of music.”
“Anyone who has never made a mistake has never tried anything new.”
“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”
“There are only two ways to live your life. One is as though nothing is a miracle. The other is as though everything is a miracle.”
“Try not to become a man of success. Rather become a man of value.”
